In [ ]:
import h5py
from ml4c3.tensormap.TensorMap import update_tmaps

Replace the dummy path below (FILE constant) by the actual path of the hd5 file that you want to extract the arrays from. 

Also you have to write the name of the tmaps. I have written those two as an example: 
* i_timeseries: returns the time series of the ecg lead I (a Bedmaster waveform). For the lead II, it would be ii_timeseries, and so on.
* art1_timeseries: returns the timeseries of the arterial pressures (a Bedmaster waveform too).
You have many other tmaps, for instance for the same signals you could extract: *i_sample_freq*, *i_units*, *i_scale_factor*
Also you have much more signals. You can get the vitalsigns derived from the art1 waveform: *art1d_timeseries* (diastolic value), *art1s_timeseries* (systolic), *art1m_timeseries* (mean). 

In [ ]:
FILE = "data/000000.hd5" # Enter the path to a file here
TMAPS = ["i_timeseries", "art1_timeseries"]

First, we load the tmaps that we want

In [ ]:

tmaps_dict = {}
for tmap_name in TMAPS:
    tmaps_dict = update_tmaps(tmap_name, tmaps_dict)


As you can see, now `tmaps_dict` has loaded the desired tmaps:

In [ ]:
tmaps_dict

Now to get the arrays on one file you just have to do:

In [ ]:
with h5py.File(FILE, "r") as hd5:
    # Extract lead I
    ecgi_tmap = tmaps_dict["i_timeseries"]
    ecgi_timeseries = ecgi_tmap.tensor_from_file(ecgi_tmap, hd5, interpolation="complete_no_nans")
    
    # Extract art
    art_tmap = tmaps_dict["art1_timeseries"]
    art_timeseries = art_tmap.tensor_from_file(art_tmap, hd5, interpolation="complete_no_nans")

The resulting array has the following shape: (n_visits, 2, n_points), where the second dimension has the values on index 0 and the timespans on index 1. Each file corresponds to an MRN so a patient could have multiple visits and ECGs could have been extracted in different visits. That's the meaning of the first dimension.

In [ ]:
ecgi_timeseries.shape

So for example, to get the values and the timespans for the first visit of that ecg:

In [ ]:
ecgi_values = ecgi_timeseries[0][0]
ecgi_time = ecgi_timeseries[0][1]